## Rotation and transposition invariant femur bone instance segmentation in CT scan

### Algorithm workflow (Python 3.8.10)
1. Preprocess and load .dcm images
2. Process images. Filter out all tissues except bones using HU scale (pixels_from=100, pixels_to=2000). HU scale highly depends on CT scanner but in most sources soft tissues have HU values below 100.
3. Using skimage.measure.label function extract largest skeletal structure
4. Filter out less dense bones and only keep bones above HU 1000. Femur bones are largest most dense bones in specified region. Pelvic bone is composed of various bone densities, therefore applying HU filter of 1000 results in smaller components of pelvic bones.
5. Skimage.measure.label is used to find 2 largest most dense structures - femur bones.
6. Padded pelvic bone and femur bone mask (created from 4. step structures) is used on processed images. Largest structure (spine and tail bones) are extracted using skimage.measure.label function.
7. Spine and tail bone orientation is used to determine which femur bone is left and right.
8. Left and right femur bones are saved in separate 3D files.


#### Function initialization

In [19]:
from pathlib import Path

import napari
import nrrd
import numpy as np
import pydicom as dicom
from scipy import ndimage


def display_3d_image(img_arr, plot_name=None, labels=None, labels_name=None):
    """Displays 3D image using Napari, displays labels if provided"""
    viewer = napari.view_image(img_arr, name=plot_name)
    if labels is not None:
        viewer.add_labels(labels, name=labels_name)
    napari.run()


def transform_to_hu(image, slope, intercept):
    """Transforms image to HU scale using .dcm metadata slope and intercept"""
    if slope != 1:
        hu_image = image * slope
    else:
        hu_image = image
    if intercept:
        hu_image += intercept
    return hu_image


def resample_image(image, resize_factor):
    """Resamples image using scipy interpolation zoom"""
    return ndimage.zoom(np.expand_dims(image, axis=2), resize_factor)


def get_important_metadata(image_list):
    """Extracts important metadata from loaded .dcm image list"""
    intercepts, slopes = zip(
        *list(map(lambda x: (int(x.RescaleIntercept), int(x.RescaleSlope)), image_list))
    )
    image_spacing = list(
        map(
            lambda x: (
                float(x.PixelSpacing[0]),
                float(x.PixelSpacing[1]),
                float(x.SliceThickness),
            ),
            image_list,
        )
    )
    image_orientation = list(map(lambda x: x.ImageOrientationPatient, image_list))
    return intercepts, slopes, image_spacing, image_orientation


def load_images(series_path, use_resample=False):
    """Loads and performs initial processing of images

    All .dcm images are loaded from specified path, important metadata and pixels are
    extracted. Images are converted to HU scale using .dcm metadata. This is important
    step since other processing step relies on HU scale for filtering out soft tissues,
    organs or metal constructions. Resampling is applied if specified.

    Args:
        series_path (Path): Loads all .dcm files from specified location.
        use_resample (bool, optional): Flag whether to use sampling. Defaults to False.
            Sampling is turned off because it does not make sense since we are working 
            only on 1 scan. 

    Returns:
        _type_: _description_
    """
    images = list(series_path.glob("*.dcm"))
    image_list = list(map(dicom.dcmread, images))
    intercepts, slopes, image_spacing, image_orientation = get_important_metadata(
        image_list
    )

    image_pixel_arr = np.array(list(map(lambda x: x.pixel_array, image_list)))

    # transform pixels to HU scale
    image_pixel_arr = np.array(
        list(map(transform_to_hu, image_pixel_arr, slopes, intercepts))
    )

    if use_resample:
        img_shape = image_pixel_arr[0].shape
        image_shape = np.array([img_shape[0], img_shape[1], 1])
        new_shapes = list(
            map(lambda x: np.round(image_shape * np.array(x)), image_spacing)
        )
        resize_factors = new_shapes / image_shape
        # resample to [1, 1, 1]
        final_images = np.array(
            list(map(resample_image, image_pixel_arr, resize_factors))
        )
    else:
        final_images = image_pixel_arr
    return final_images.squeeze()


def save_left_and_right_femur(processed_image_arr, right_femur_mask, left_femur_mask):
    """Saves right and left femurs to separate 3D files"""
    final_mask = processed_image_arr.copy()
    final_mask = np.array(final_mask != 0, dtype=int)
    right_femur_labeled_ind = (right_femur_mask) & (final_mask)
    left_femur_labeled_ind = (left_femur_mask) & (final_mask)
    nrrd.write("right_femur.nrrd", right_femur_labeled_ind)
    nrrd.write("left_femur.nrrd", left_femur_labeled_ind)


In [20]:
from collections import Counter

import scipy
from scipy.ndimage import binary_fill_holes, binary_dilation
from skimage import filters, measure, morphology
from skimage.morphology import binary_erosion, disk


def get_image_filter_mask(image_arr, pixels_from, pixels_to):
    """Returns index values where image pixels are between `pixels_from` and `pixels_to`"""
    shape = image_arr.shape
    ind1 = image_arr >= pixels_from if pixels_from else np.ones(shape, dtype=bool)
    ind2 = image_arr <= pixels_to if pixels_to else np.ones(shape, dtype=bool)
    ind = (ind1) & (ind2)
    return ind


def get_labeled_components(mask, keep_n_biggest_objects=1):
    """Uses skimage measure module to find all structures in 3D image and returns specified
        number of largest objects.

    Args:
        mask (np.array): boolean or integer 3D image mask.
        keep_n_biggest_objects (int, optional): Saves specified number of biggest objects.
            If set to 0, returns all objects. Defaults to 1.

    Returns:
        All labels and list of biggest object labels

    """
    # 0 - keep all,
    labels = measure.label(mask)
    counter_dict = Counter(labels.flatten())
    objects = counter_dict.most_common()
    keep_objects = (
        objects[1 : keep_n_biggest_objects + 1] if keep_n_biggest_objects else objects
    )
    return labels, keep_objects


def perform_erosion(image_arr, **kwargs):
    """Performs erosion on image"""
    mask = image_arr != 0
    binary_img = binary_erosion(mask, **kwargs)
    image_arr[~binary_img] = 0
    return image_arr


def remove_small_objects_and_holes(
    image_arr, remove_holes=False, threshold_type="li", min_size=300
):
    """Removes small objects and holes from image"""
    find_threshold = (
        filters.threshold_li if threshold_type == "li" else filters.threshold_otsu
    )
    tgray = image_arr > find_threshold(image_arr)
    keep_mask = morphology.remove_small_objects(tgray, min_size=min_size)
    if remove_holes:
        keep_mask = morphology.remove_small_holes(keep_mask)
    image_arr[~keep_mask] = 0
    return image_arr, keep_mask


def keep_labeled_components(image_arr, keep_structures, labels):
    """Filters out all other structures in image except specified structures"""
    image_arr_copy = image_arr.copy()
    keep_objects_dict = dict(keep_structures)
    ind = np.zeros(image_arr_copy.shape)
    for i in keep_objects_dict.keys():
        ind = (ind != 0) | (labels == i)
    image_arr_copy[~ind] = 0
    return image_arr_copy


def image_processing(
    image_pixel_arr, pixels_from=100, pixels_to=2000, remove_small_objects_size=300
):
    """Processes loaded images by filtering out all other structures except bones.

    Uses pixel filtering, erosion, removal of small objects and skimage measure 
    label function. Label function is used to keep biggest bone structure in image
    while filtering out CT scan screen, metals rods and etc.

    Args:
        image_pixel_arr (np.array): Image array.
        pixels_from (int, optional): Pixels to keep in image `from` limit. Defaults to 100.
        pixels_to (int, optional): Pixels to keep in image `to` limit. Defaults to 2000.
        remove_small_objects_size (int, optional): Pixel size of objects which are removed.
            Defaults to 300.

    Returns:
        Numpy array Image Bone structure

    """
    # removes larger part of rods but leaves artifacts
    image_pixel_arr_copy = image_pixel_arr.copy()

    # removing soft tissues and other noise
    ind = get_image_filter_mask(image_pixel_arr, pixels_from, pixels_to)
    image_pixel_arr_copy[~ind] = 0

    # decreasing size of artifacts
    image_pixel_arr_copy = perform_erosion(image_pixel_arr_copy)

    # removing small objects
    image_pixel_arr_copy, keep_mask = remove_small_objects_and_holes(
        image_pixel_arr_copy, min_size=remove_small_objects_size
    )

    # Filtering out noise by keeping only largest (bone) structure
    labels, keep_objects = get_labeled_components(keep_mask, 1)
    image_pixel_arr_copy = keep_labeled_components(
        image_pixel_arr_copy, keep_objects, labels
    )
    return image_pixel_arr_copy, keep_mask


def generate_3d_mask_rectangular(
    mindim0, maxdim0, mindim1, maxdim1, mindim2, maxdim2, image_size, padding=0
):
    """Generates 3D mask with a rectangle shape based on points"""
    mask_3d = np.zeros(image_size, dtype=int)

    if padding:
        maxdim0 += padding
        maxdim1 += padding
        maxdim2 += padding
        mindim0 = mindim0 - padding if mindim0 - padding > 0 else mindim0
        mindim1 = mindim1 - padding if mindim1 - padding > 0 else mindim1
        mindim2 = mindim2 - padding if mindim2 - padding > 0 else mindim2

    mask_3d[mindim0:maxdim0, mindim1:maxdim1, mindim2:maxdim2] = 1
    return mask_3d


def get_dimensions_for_nd_mask(image_arr):
    """Calculates object points from image mask"""
    # returns first, second, third dimension min/max values
    r = np.argwhere(image_arr != 0)
    min_points = r.min(axis=0)
    max_points = r.max(axis=0)
    return np.stack((min_points, max_points), axis=1)


def change_axis_order_or_rotate(image_arr, rotate_angle=0, axis_order=[0, 1, 2]):
    """Changes 3D image axis order or rotation angle
        Used for testing the algorithm"""
    if axis_order != [0, 1, 2]:
        image_arr = image_arr.transpose(axis_order)
    if rotate_angle != 0 or rotate_angle % 360 != 0:
        image_arr = scipy.ndimage.rotate(
            image_arr, angle=rotate_angle, reshape=False, axes=(1, 2)
        )
    return image_arr


#### Separating structures and keeping only bone structure

In [21]:
series_path = Path('klubo_atvejis')

image_pixel_arr = load_images(series_path)


In [22]:
processed_image_arr, keep_mask = image_processing(image_pixel_arr)

In [23]:
display_3d_image(processed_image_arr)

#### Extracting and separating femur, pelvic bones, creating 3D masks

In [24]:
image_pixel_arr_copy = processed_image_arr.copy()

# Keeping structures with HU > 1000
keep_ind = get_image_filter_mask(image_pixel_arr_copy, 1000, None)
image_pixel_arr_copy[~keep_ind] = 0

# Performing erosion and removal of small objects and holes
image_pixel_arr_copy = perform_erosion(image_pixel_arr_copy)
image_pixel_arr_copy, mask = remove_small_objects_and_holes(
    image_pixel_arr_copy, remove_holes=True, threshold_type="other", min_size=400
)

# Extracting components
labels, keep_objects = get_labeled_components(mask, keep_n_biggest_objects=0)


In [25]:
# Femur bones are largest from high density bones
femur_bones_combined = keep_labeled_components(
    image_pixel_arr_copy, keep_objects[1:3], labels
)

# Extracting separate structures of femur bones and pelvic bones
femur_bone_one = keep_labeled_components(image_pixel_arr_copy, keep_objects[1:2], labels)
femur_bone_two = keep_labeled_components(image_pixel_arr_copy, keep_objects[2:3], labels)
pelvic_bones = keep_labeled_components(image_pixel_arr_copy, keep_objects[3:], labels)


In [26]:
# Creting pelvic 3D mask and padding it
series_shape = pelvic_bones.shape
pelvic_bones_2d_mask = pelvic_bones.mean(axis=0) != 0
pelvic_3d_mask = np.tile(pelvic_bones_2d_mask, (series_shape[0], 1, 1))
filled_holes = binary_fill_holes(pelvic_3d_mask[0, :, :])
radius = 20
filled_holes = binary_dilation(filled_holes, disk(radius, dtype=bool))
pelvis_3d_mask_dilated = np.tile(filled_holes, (series_shape[0], 1, 1))


In [27]:
# creating femur bones combined rectangle mask 
(mindim0, maxdim0), (mindim1, maxdim1), (mindim2, maxdim2) = get_dimensions_for_nd_mask(
    femur_bones_combined
)
generated_femurs_combined_mask = generate_3d_mask_rectangular(
    mindim0, maxdim0, mindim1, maxdim1, mindim2, maxdim2, series_shape, padding=20
)

# creating femur one rectangle bone mask
(
    (mindim0_f1, maxdim0_f1),
    (mindim1_f1, maxdim1_f1),
    (mindim2_f1, maxdim2_f1),
) = get_dimensions_for_nd_mask(femur_bone_one)

generated_femur_one_mask = generate_3d_mask_rectangular(
    mindim0_f1,
    maxdim0_f1,
    mindim1_f1,
    maxdim1_f1,
    mindim2_f1,
    maxdim2_f1,
    series_shape,
    padding=10,
)

# creating femur two rectangle bone mask
(
    (mindim0_f2, maxdim0_f2),
    (mindim1_f2, maxdim1_f2),
    (mindim2_f2, maxdim2_f2),
) = get_dimensions_for_nd_mask(femur_bone_two)

generated_femur_two_mask = generate_3d_mask_rectangular(
    mindim0_f2,
    maxdim0_f2,
    mindim1_f2,
    maxdim1_f2,
    mindim2_f2,
    maxdim2_f2,
    series_shape,
    padding=10,
)


#### Finding spine and tail bones from processed images. Pelvic and femur bones were removed from processed images. Spine and tail bones are found using skimage.measure.label since it is the biggest remaining structure

In [28]:
pelvis_ind_and_femur_mask = (pelvis_3d_mask_dilated != 0) | (
    generated_femurs_combined_mask
)
filtered_out_pelvis_and_femur = processed_image_arr.copy()
filtered_out_pelvis_and_femur[pelvis_ind_and_femur_mask != 0] = 0


In [29]:
# Finding largest structure (spine)
labels, keep_objects = get_labeled_components(
    filtered_out_pelvis_and_femur != 0, keep_n_biggest_objects=1
)

spine_bone = keep_labeled_components(filtered_out_pelvis_and_femur, keep_objects, labels)


In [30]:
# Generating spine rectangle mask
(
    (mindim0_s, maxdim0_s),
    (mindim1_s, maxdim1_s),
    (mindim2_s, maxdim2_s),
) = get_dimensions_for_nd_mask(spine_bone)

generated_spine_mask = generate_3d_mask_rectangular(
    mindim0_s, maxdim0_s, mindim1_s, maxdim1_s, mindim2_s, maxdim2_s, series_shape
)


#### Finding femur positions relative to spine. Separating femurs to left and right

In [31]:
# femur 1 mid point
mid_dim0_f1_p = (mindim0_f1 + maxdim0_f1) / 2
mid_dim1_f1_p = (mindim1_f1 + maxdim1_f1) / 2
mid_dim2_f1_p = (mindim2_f1 + maxdim2_f1) / 2
femur_1_mid_point = np.array([mid_dim0_f1_p, mid_dim1_f1_p, mid_dim2_f1_p])

# femur 2 mid point
mid_dim0_f2_p = (mindim0_f2 + maxdim0_f2) / 2
mid_dim1_f2_p = (mindim1_f2 + maxdim1_f2) / 2
mid_dim2_f2_p = (mindim2_f2 + maxdim2_f2) / 2
femur_2_mid_point = np.array([mid_dim0_f2_p, mid_dim1_f2_p, mid_dim2_f2_p])

# spine mid point
mindim0_s_p = (mindim0_s + maxdim0_s) / 2
mindim1_s_p = (mindim1_s + maxdim1_s) / 2
mindim2_s_p = (mindim2_s + maxdim2_s) / 2
spine_mid_point = np.array([mindim0_s_p, mindim1_s_p, mindim2_s_p])
femur_1_mid_point, femur_2_mid_point, spine_mid_point

(array([ 49. , 277.5, 427. ]),
 array([ 40.5, 310.5, 135. ]),
 array([216.5, 317. , 282. ]))

In [32]:
# Calculate vectors from C to A and C to B
vector_CA = femur_1_mid_point - spine_mid_point
vector_CB = femur_2_mid_point - spine_mid_point

cross_product_z = np.cross(vector_CA, vector_CB)

In [33]:
# Since .DCM images have the same number of columns and rows, we can know where is depth dimensions: https://dicom.nema.org/medical/Dicom/2018d/output/chtml/part03/sect_C.7.6.3.html
# Finding which femur is on the right side of spine and tail bone
depth_dim_location = (
    0 if len(image_pixel_arr) == generated_spine_mask.shape[0] else -1
)
if depth_dim_location != 0:
    right_femur_mask, left_femur_mask = (
        (generated_femur_two_mask, generated_femur_one_mask)
        if cross_product_z[0] < 0
        else (generated_femur_one_mask, generated_femur_two_mask)
    )
else:
    right_femur_mask, left_femur_mask = (
        (generated_femur_two_mask, generated_femur_one_mask)
        if cross_product_z[0] > 0
        else (generated_femur_one_mask, generated_femur_two_mask)
    )


#### Saving results to separate 3D files

In [34]:
save_left_and_right_femur(processed_image_arr, right_femur_mask, left_femur_mask)

#### Displaying labeled femur bones on processed skeleton

In [35]:
final_mask = processed_image_arr.copy()
final_mask = np.array(final_mask != 0, dtype=int)
right_femur_labeled_ind = (right_femur_mask) & (final_mask)
left_femur_labeled_ind = (left_femur_mask) & (final_mask)
left_femur_labeled_ind[left_femur_labeled_ind!=0] += 1

In [36]:
viewer = napari.view_image(processed_image_arr)
viewer.add_labels(right_femur_labeled_ind, name='right')
viewer.add_labels(left_femur_labeled_ind, name='left')

<Labels layer 'left' at 0x19f00e26b20>

## Segmentation can be improved by multiple ways:
1. Adding sphere mask on top of femur rectangle mask
2. Filtering out pelvic bones (based on pelvic rectangle mask) which are above femur mask + some buffer zone. Then use measure.label to find 2 largest structures (femur bones) 